<a href="https://colab.research.google.com/github/freguti/Homework-ML/blob/Homework_3/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Install requirements**

In [81]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

**Import libraries**

In [0]:
import os
import logging
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm
from copy import deepcopy

**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 7 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.005            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 20      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 8       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LAMBDA = 0.1
LOG_FREQUENCY = 1

**Train without DANN**

In [0]:
def train(model, train_dataloader):
  # Define loss function
  criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

  # Choose parameters to optimize
  # To access a different set of parameters, you have to access submodules of AlexNet
  # (nn.Module objects, like AlexNet, implement the Composite Pattern)
  # e.g.: parameters of the fully connected layers: net.classifier.parameters()
  # e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
  parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet

  # Define optimizer
  # An optimizer updates the weights based on loss
  # We use SGD with momentum
  optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

  # Define scheduler
  # A scheduler dynamically changes learning rate
  # The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

  # By default, everything is loaded to cpu
  model = model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

  cudnn.benchmark # Calling this optimizes runtime

  current_step = 0
  # Start iterating over the epochs
  for epoch in range(NUM_EPOCHS):
    print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

    # Iterate over the dataset
    n_loss = 0
    current_step = 0
    for images, labels, domains in train_dataloader:
      # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)
      domains = domains.to(DEVICE)

      model.train() # Sets module in training mode

      # PyTorch, by default, accumulates gradients after each backward pass
      # We need to manually set the gradients to zero before starting a new iteration
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = model(images)

      # Compute loss based on output and ground truth
      loss = criterion(outputs, labels)

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients
      n_loss += loss.item()
      current_step += 1
    n_loss = n_loss/current_step
    print('Epoch {},Medium Loss {}'.format(epoch+1, n_loss))
    # Step the scheduler
    scheduler.step() 

In [0]:
def batch_it(dataloader_iterator,taget_dataloader):
  for i in range(len(taget_dataloader)):
    try:
      data = next(dataloader_iterator)
    except StopIteration:
      dataloader_iterator = iter(taget_dataloader)
      data = next(dataloader_iterator)
  return data[0],data[1] #data,target

**Train with DANN**

In [0]:
def DANN_train(model, train_dataloader,test_dataloader, lrs = [LR], STEPs = [STEP_SIZE], Ns = [NUM_EPOCHS], LAMBDAs = [LAMBDA]):
  params = {'LR' : [], 'STEP_SIZE' : [], 'NUMBER_EPOCHS' : [], 'LAMBDA' : [], 'accuracy' : [], 'loss' : [],'domain_loss' : [],'test_domain_loss' : []}
  for lr,step_size,num_epoch,v_lambda in zip(lrs,STEPs,Ns,LAMBDAs):
    print('lr {} step {} epochs {} lambda {}'.format(lr,step_size,num_epoch,v_lambda))
    params['LR'].append(lr)
    params['STEP_SIZE'].append(step_size)
    params['NUMBER_EPOCHS'].append(num_epoch)
    params['LAMBDA'].append(v_lambda)
  # Define loss function
    criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

    # Choose parameters to optimize
    # To access a different set of parameters, you have to access submodules of AlexNet
    # (nn.Module objects, like AlexNet, implement the Composite Pattern)
    # e.g.: parameters of the fully connected layers: net.classifier.parameters()
    # e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
    parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet

    # Define optimizer
    # An optimizer updates the weights based on loss
    # We use SGD with momentum
    optimizer = optim.SGD(parameters_to_optimize, lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

    # Define scheduler
    # A scheduler dynamically changes learning rate
    # The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=GAMMA)

    # By default, everything is loaded to cpu
    model = model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

    cudnn.benchmark # Calling this optimizes runtime

    current_step = 0
    dataloader_iterator = iter(test_dataloader)
    # Start iterating over the epochs
    for epoch in range(num_epoch):
      print('Starting epoch {}/{}, LR = {}'.format(epoch+1, num_epoch, scheduler.get_lr()))
      n_loss = 0
      domainloss = 0
      t_domainloss = 0
      current_step = 0
      # Iterate over the dataset
      for images, labels, domains in train_dataloader:
        # Bring data over the device of choice
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        domains = domains.to(DEVICE)

        model.train() # Sets module in training mode

        # PyTorch, by default, accumulates gradients after each backward pass
        # We need to manually set the gradients to zero before starting a new iteration
        optimizer.zero_grad() # Zero-ing the gradients
        # Forward pass to the network
        outputs = model(images)
        # Compute loss based on output and ground truth
        loss = criterion(outputs, labels)
        # Compute gradients for each layer and update weights
        loss.backward()  # update gradients with loss.backward

        n_loss += loss.item()
        #training discriminator by forwarding source data
        outputs_discriminator = model.forward(images,v_lambda)
        targ = torch.zeros(labels.size(0), dtype = torch.int64).to(DEVICE)
        loss_discriminator = criterion(outputs_discriminator,targ)
        loss_discriminator.backward()

        domainloss += loss_discriminator.item()
        optimizer.step() # update weights based on accumulated gradients

        #load batch from test dataloader. For every train batch I have to take a test batch 
        images_test,label_test = batch_it(dataloader_iterator,test_dataloader)
        images_test = images_test.to(DEVICE)
        label_test = label_test.to(DEVICE)
        outputs_discriminator_test = model.forward(images_test,v_lambda)
        targ = torch.ones(labels.size(0), dtype = torch.int64).to(DEVICE)
        loss_discriminator_test = criterion(outputs_discriminator_test,targ)
        loss_discriminator_test.backward()   

        t_domainloss += loss_discriminator_test.item()


        current_step += 1
      n_loss = n_loss/current_step
      domainloss = domainloss/current_step
      t_domainloss = t_domainloss/current_step
      print('Epoch {},Medium Loss {}, Domain Loss (?) {}, Test domain loss {} '.format(epoch+1, n_loss, domainloss, t_domainloss))
      # Step the scheduler
      scheduler.step()
    params['loss'].append(n_loss)
    params['domain_loss'].append(domainloss)
    params['test_domain_loss'].append(t_domainloss)
  return params

**Alex Net**

In [0]:
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url

try:
  from torch.hub import load_state_dict_from_url
except ImportError:
  from torch.utils.model_zoo import load_url as load_state_dict_from_url

__all__ = ['AlexNet', 'alexnet']
model_urls = { 'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth', }

from torch.autograd import Function


class ReverseLayerF(Function):
    # Forwards identity
    # Sends backward reversed gradients
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        self.domainclassifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2),
        )

    def forward(self, x, alpha=None):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        if alpha is not None:
          # gradient reversal layer (backward gradients will be reversed)
         reverse_feature = ReverseLayerF.apply(x, alpha)
         discriminator_output = self.domainclassifier(reverse_feature)
         return discriminator_output
        # If we don't pass alpha, we assume we are training with supervision
        else:
          # do something else
          class_outputs = self.classifier(x)
          return class_outputs
      

def alexnet(pretrained=False, progress=True, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict, strict=False)
    return model

In [0]:

from torchvision.datasets import VisionDataset
import torchvision.transforms.functional as F

import os.path
import sys

DATA_DIR = 'Homework3-PACS/PACS'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class Caltech(VisionDataset):
    def __init__(self, transform=None, target_transform=None, sets = []):
        super(Caltech, self).__init__(root = DATA_DIR, transform=transform, target_transform=target_transform)
        d = 0
        self.sets = sets
        self.domain = []
        for label in ['art_painting','cartoon','photo','sketch']:
          self.domain.append((label,d))
          d = d + 1
        self.dataset = self.read()
        
        return

    def read(self):
      fine_dataset = []
      
      for idx in self.sets:
        #print(label)
        label = self.domain[idx]
        print(DATA_DIR + '/' + label[0])
        my_dataset = torchvision.datasets.ImageFolder(DATA_DIR + '/' + label[0], transform=train_transform)
        for element in my_dataset:
          fine_dataset.append((F.to_pil_image(element[0]),element[1],label[1])) 
      #print("________________________________")
      #print(fine_dataset[0]) #1) [] mi indica il numero di elemento
                                #2) [] mi indica la tupla (tensor,class)
                                #3) [] mi indica l'elemento della tupla
                                #TODO SPLIT
      del my_dataset
      return fine_dataset

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''
        image,label,domain = self.dataset[index]

        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)
        return image, label, domain

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.dataset) # Provide a way to get the length (number of elements) of the dataset
        return length


**Define Data Preprocessing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

**Clone Github repository**

In [0]:
# Clone github repository with data
if not os.path.isdir('./Homework3-PACS'):
  !git clone https://github.com/MachineLearning2020/Homework3-PACS



**Prepare Dataset**

In [91]:
train_dataset = Caltech(train_transform,sets = [2])
test_dataset = Caltech(eval_transform,sets = [0])

#1/5 in test and 4/5 in train
#train_indexes = [idx for idx in range(len(train_dataset)) if idx % 5]
#test_indexes = [idx for idx in range(len(test_dataset)) if not idx % 5]

#train_dataset = Subset(train_dataset, train_indexes)
#test_dataset = Subset(test_dataset, test_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Homework3-PACS/PACS/photo
Homework3-PACS/PACS/art_painting
Train Dataset: 1670
Test Dataset: 2048


**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare Network**

In [0]:
net = alexnet(pretrained = True) # Loading AlexNet model
#copia dei pesi
net.classifier[6] = nn.Linear(4096, 7)
weight1=deepcopy(net.classifier[1].weight.data)
bias1=deepcopy(net.classifier[1].bias.data)
weight4=deepcopy(net.classifier[4].weight.data)
bias4=deepcopy(net.classifier[4].bias.data)
net.domainclassifier[1].weight.data=weight1
net.domainclassifier[1].bias.data=bias1
net.domainclassifier[4].weight.data=weight4
net.domainclassifier[4].bias.data=bias4
# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is mandatory to study torchvision.models.alexnet source code

**Prepare Training**

In [0]:
# Define loss function
#criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
#parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
#optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train**

In [95]:
train(net,train_dataloader)

Starting epoch 1/20, LR = [0.005]
Epoch 1,Medium Loss 0.8764349451431861
Starting epoch 2/20, LR = [0.005]
Epoch 2,Medium Loss 0.4483634508573092
Starting epoch 3/20, LR = [0.005]
Epoch 3,Medium Loss 0.28212379262997556
Starting epoch 4/20, LR = [0.005]
Epoch 4,Medium Loss 0.1888432691876705
Starting epoch 5/20, LR = [0.005]
Epoch 5,Medium Loss 0.10638044201410733
Starting epoch 6/20, LR = [0.005]
Epoch 6,Medium Loss 0.08652163125001468
Starting epoch 7/20, LR = [0.005]
Epoch 7,Medium Loss 0.07621298701717304
Starting epoch 8/20, LR = [0.005]
Epoch 8,Medium Loss 0.06320543644519952
Starting epoch 9/20, LR = [0.0005]
Epoch 9,Medium Loss 0.04003281862689899
Starting epoch 10/20, LR = [0.0005]
Epoch 10,Medium Loss 0.020376640157057688
Starting epoch 11/20, LR = [0.0005]
Epoch 11,Medium Loss 0.016517190549236078
Starting epoch 12/20, LR = [0.0005]
Epoch 12,Medium Loss 0.014881056375228442
Starting epoch 13/20, LR = [0.0005]
Epoch 13,Medium Loss 0.014192543637294035
Starting epoch 14/20, LR

**Test**

In [96]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels, domains in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)
  domains = domains.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 16/16 [00:03<00:00,  4.16it/s]

Test Accuracy: 0.42138671875


In [97]:
net = alexnet(pretrained = True) # Loading AlexNet model
#copia dei pesi
net.classifier[6] = nn.Linear(4096, 7)
weight1=deepcopy(net.classifier[1].weight.data)
bias1=deepcopy(net.classifier[1].bias.data)
weight4=deepcopy(net.classifier[4].weight.data)
bias4=deepcopy(net.classifier[4].bias.data)
net.domainclassifier[1].weight.data=weight1
net.domainclassifier[1].bias.data=bias1
net.domainclassifier[4].weight.data=weight4
net.domainclassifier[4].bias.data=bias4
# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is mandatory to study torchvision.models.alexnet source code

params = DANN_train(net, train_dataloader,test_dataloader)
print(params)

lr 0.005 step 8 epochs 20 lambda 0.1
Starting epoch 1/20, LR = [0.005]
Epoch 1,Medium Loss 0.937473136645097, Domain Loss (?) 0.05756676433464655, Test domain loss 27.686811960660496 
Starting epoch 2/20, LR = [0.005]
Epoch 2,Medium Loss 0.4055476486682892, Domain Loss (?) 3.69949982716487e-07, Test domain loss 29.754217587984524 
Starting epoch 3/20, LR = [0.005]
Epoch 3,Medium Loss 0.23539673823576707, Domain Loss (?) 8.195638656616211e-08, Test domain loss 31.913848583514874 
Starting epoch 4/20, LR = [0.005]
Epoch 4,Medium Loss 0.148233100771904, Domain Loss (?) 5.73121584378756e-10, Test domain loss 42.74860822237455 
Starting epoch 5/20, LR = [0.005]
Epoch 5,Medium Loss 0.10448712110519409, Domain Loss (?) 2.86560792189378e-10, Test domain loss 42.31688455434946 
Starting epoch 6/20, LR = [0.005]
Epoch 6,Medium Loss 0.06257180726298919, Domain Loss (?) 5.73121584378756e-10, Test domain loss 48.969573094294624 
Starting epoch 7/20, LR = [0.005]
Epoch 7,Medium Loss 0.06703542946622

In [98]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels, domains in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)
  domains = domains.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 16/16 [00:03<00:00,  4.22it/s]

Test Accuracy: 0.4111328125


**CROSS DOMAIN VALIDATION (GRID SEARCH)**

In [100]:
#TODO: ok, ma deve essere fatto senza adaptation, vedere chat con ivan

#cross domain validation
lrs =[0.001,0.01,0.05]
STEPs = [6,10,15] 
Ns =[10, 15, 20]
LAMBDAs =[0.05,0.1,0.2]

net = alexnet(pretrained = True) # Loading AlexNet model
#copia dei pesi
net.classifier[6] = nn.Linear(4096, 7)
weight1=deepcopy(net.classifier[1].weight.data)
bias1=deepcopy(net.classifier[1].bias.data)
weight4=deepcopy(net.classifier[4].weight.data)
bias4=deepcopy(net.classifier[4].bias.data)
net.domainclassifier[1].weight.data=weight1
net.domainclassifier[1].bias.data=bias1
net.domainclassifier[4].weight.data=weight4
net.domainclassifier[4].bias.data=bias4
# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is mandatory to study torchvision.models.alexnet source code

params = DANN_train(net, train_dataloader,test_dataloader,lrs,STEPs,Ns,LAMBDAs)
print(params)

lr 0.001 step 6 epochs 10 lambda 0.05
Starting epoch 1/10, LR = [0.001]
Epoch 1,Medium Loss 1.2069829198030324, Domain Loss (?) 0.051870988932653114, Test domain loss 9.341168128527128 
Starting epoch 2/10, LR = [0.001]
Epoch 2,Medium Loss 0.5254485882245578, Domain Loss (?) 6.317497732547614e-05, Test domain loss 20.432390726529636 
Starting epoch 3/10, LR = [0.001]
Epoch 3,Medium Loss 0.3909758237692026, Domain Loss (?) 2.8494029090954707e-05, Test domain loss 23.996039463923527 
Starting epoch 4/10, LR = [0.001]
Epoch 4,Medium Loss 0.3112450929788443, Domain Loss (?) 3.240565554453777e-05, Test domain loss 22.4112371297983 
Starting epoch 5/10, LR = [0.001]
Epoch 5,Medium Loss 0.26406027720524716, Domain Loss (?) 2.5764394264954786e-05, Test domain loss 21.754115764911358 
Starting epoch 6/10, LR = [0.001]
Epoch 6,Medium Loss 0.21926149496665367, Domain Loss (?) 2.4965032935142517e-05, Test domain loss 22.727765743549053 
Starting epoch 7/10, LR = [0.0001]
Epoch 7,Medium Loss 0.1891

KeyboardInterrupt: ignored